# The Device

In [1]:
import torch

# GPU
if torch.cuda.is_available():
    device = torch.device(f"cuda:{0}")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

We will use the GPU: Quadro RTX 8000


# utils

In [2]:
import numpy as np

NUM_LABELS = {
    "ana": 2,
    "dna": 2,
    "dnaa": 2,
    "rpsv": 2,
    "darn": 2,
    "NA": 2,
}

blimp_to_label = {
    'singular': 0,
    'plural': 1,
}

MODEL_PATH = {
    'bert': 'bert-base-uncased',
    'roberta': 'roberta-base',
    'electra': 'google/electra-base-generator',
    'deberta': 'microsoft/deberta-v3-base'
}

BLIMP_TASKS = [
    "ana",
    'dna',
    "dnaa",
    "rpsv",
    "darn",
    "NA",
]

def blimp_to_features(data, tokenizer, max_length, input_masking, mlm):
    all_features = []
    for example in data:
        text = example['sentence_good']
        tokens = []
        cue_indices = []
        # token to id
        for w_ind, word in enumerate(text):
            ids = tokenizer.encode(word, add_special_tokens=False)
            if w_ind in example['cue_indices']:
                cue_indices.append(len(tokens))
            if w_ind == example['target_index']:
                target_index = len(tokens)
            tokens.extend(ids)
        
        tokens = [tokenizer.cls_token_id] + tokens + [tokenizer.sep_token_id]
        cue_indices = [x+1 for x in cue_indices] # 'cause of adding cls
        target_index += 1 # 'cause of adding cls
        if input_masking:
            tokens[target_index] = tokenizer.mask_token_id

        # padding
        length = len(tokens)
        inputs = {}
        inputs['input_ids'] = tokens if max_length is None else tokens + [tokenizer.pad_token_id]*(max_length - length)
        inputs['input_ids'] = torch.tensor(inputs['input_ids']).to(device) # .unsqueeze(0)
        inputs['attention_mask'] = [1]*length if max_length is None else [1]*length + [0]*(max_length - length)
        inputs['attention_mask'] = torch.tensor(inputs['attention_mask']).to(device) # .unsqueeze(0).
        # inputs['token_type_ids'] = [0]*length if max_length is None else [0]*max_length
        inputs['target_index'] = target_index

        # As a 2d tensor, we need all rows to have the same length. So, we add -1 to the end of each list.
        inputs['cue_indices'] = cue_indices + (10 - len(cue_indices)) * [-1]

        all_features.append(inputs)
    return all_features[0] if len(all_features) == 1 else all_features

PREPROCESS_FUNC = {
    'ana': blimp_to_features,
    'dna': blimp_to_features,
    'dnaa': blimp_to_features,
    'rpsv': blimp_to_features,
    'darn': blimp_to_features,
    'NA': blimp_to_features,
}

In [3]:
from src.utils_contributions import *
import torch.nn.functional as F
from src.contributions import ModelWrapper, ClassificationModelWrapperCaptum, interpret_sentence, occlusion
import pandas as pd
import seaborn as sns
import json
import random
random.seed(10)

from collections import defaultdict

import torch.nn as nn

In [4]:
SPLIT = 'test'
MODEL_NAME = 'roberta'

data_path = f"./BLIMP Dataset/{MODEL_NAME}/"
dataset = load_from_disk(data_path)[SPLIT]

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH[MODEL_NAME])
config = AutoConfig.from_pretrained(MODEL_PATH[MODEL_NAME], num_labels=2)
model = AutoModelForMaskedLM.from_pretrained(MODEL_PATH[MODEL_NAME], config=config)

In [5]:
model.load_state_dict(torch.load('./roberta_full_forseqclassification_finetuned_MLM_epoch1.pt'))

<All keys matched successfully>

In [21]:
SELECTED_GPU = 0
MODEL_NAME = 'roberta'
FIXED = False
TASK = "NA"
MAX_LENGTH = None
NUM_TRAIN_EPOCHS = 1
PER_DEVICE_BATCH_SIZE = 1

INPUT_MASKING = True
MLM = True
LEARNING_RATE = 3e-5
LR_SCHEDULER_TYPE = "linear" 
WARMUP_RATIO = 0.1
SEED = 42

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AdamW,
    get_scheduler,
    default_data_collator,
    set_seed,
)
from torch.utils.data import DataLoader

eval_dataset = PREPROCESS_FUNC[TASK](dataset, tokenizer, MAX_LENGTH, input_masking=INPUT_MASKING, mlm=MLM)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=PER_DEVICE_BATCH_SIZE)

In [7]:
model = model.to(device)

# ALTI

The implementations were sourced from https://github.com/mt-upc/transformer-contributions.

In [24]:
from tqdm.auto import tqdm

keys_to_select = ['input_ids', 'attention_mask']
model_wrapped = ModelWrapper(model)

ALTI_per_layer_scores = list()
for batch_sample in tqdm(eval_dataloader):
    
    subset_dict = {key: batch_sample[key] for key in keys_to_select if key in batch_sample}
    pt_batch = subset_dict
    
    prediction_scores, hidden_states, attentions, contributions_data = model_wrapped(pt_batch)
    probs = torch.nn.functional.softmax(prediction_scores, dim=-1)
    pred_ind = torch.argmax(probs)
    pred = torch.max(probs)
    
    _attentions = [att.detach().cpu().numpy() for att in attentions]
    attentions_mat = np.asarray(_attentions)[:,0] # (num_layers,num_heads,src_len,src_len)
    att_mat_sum_heads = attentions_mat.sum(axis=1) / attentions_mat.shape[1]
    normalized_model_norms = normalize_contributions(contributions_data['transformed_vectors_norm'],scaling='sum_one')
    resultant_norm = resultants_norm = torch.norm(torch.squeeze(contributions_data['resultants']),p=1,dim=-1)
    # ALTI Requires scaling = min_sum
    normalized_contributions = normalize_contributions(contributions_data['contributions'], scaling='min_sum', resultant_norm=resultant_norm)
    
    ALTI_per_layer_scores.append(normalized_contributions)
    

  0%|          | 0/2208 [00:00<?, ?it/s]

# Alignment Metrics

Here, we compute Dot Product and Average Precision. At first, let's define a method to compute Average Precision.

In [25]:
# Cleaning the tensors from -1 padding
def CI_cleaner(CI):
    first_pad_index = torch.where(CI == -1)[0][0].item() # We have used -1 as paddings of CIs
    return CI[:first_pad_index]

# Calculating Precision
def precision(TP, FP):
    return TP / (TP + FP)

# Calculating Recall
def recall(TP, FN):
    return TP / (TP + FN)

# Calculating Average Precision
def avg_precision(topk, CI):
    R_base = 0 # The starting recall before the first round
    AP, TP, FP, FN = 0, 0, 0, len(CI)
    previous_recall = R_base
    for i in range(len(topk)):
        if topk[i] in CI:
            TP += 1
            FN -= 1
        else:
            FP += 1

        AP += (recall(TP, FN) -  previous_recall) * precision(TP, FP)
        previous_recall = recall(TP, FN)

    return AP

topk = torch.tensor([1, 0, 3, 4, 2])
CI = torch.tensor([3, -1, -1, -1])
# CI = CI_cleaner(CI)
# print(avg_precision(topk, CI))

In [26]:
diagram_layers = range(12)
APs_ALTI = dict()

for layer in diagram_layers:
    APs_ALTI[f'layer{layer}'] = list()

sum_ALTI_scores = 0

model.eval()
for i, batch_sample in enumerate(tqdm(eval_dataset)):
    
    CI = CI_cleaner(torch.tensor(batch_sample['cue_indices'])) # [0]: because we only have one sample in each batch
    
    ### Average Precision
    batch_lengths = batch_sample['attention_mask'].sum(axis=-1)
    mask_index = batch_sample['target_index'] # mask_index = target_index

    # The contribution of each token in the sequence in building the rep. of target token for different layers
    ALTI_importance = ALTI_per_layer_scores[i][:, batch_sample['target_index']] # shape: [12, seq_len]
    # Convert to torch tensor form numpy ndarray
#     ALTI_importance = torch.from_numpy(ALTI_importance)
    # batch_lengths[0]: because we only have one sample in each batch
    ALTI_importance_topk = torch.topk(ALTI_importance, k=batch_lengths.item(), largest=True, dim=1).indices
    
    ### excluding mask_index
    mask_index_tensor = torch.full_like(ALTI_importance_topk, mask_index)
    # Create a mask that is True for elements not equal to mask_index
    mask = ALTI_importance_topk != mask_index_tensor
    # # Apply the mask to exclude mask_index
    ALTI_importance_topk_filtered = ALTI_importance_topk[mask].view(ALTI_importance_topk.size(0), -1)

    for layer, layer_importance in enumerate(ALTI_importance_topk_filtered):
        APs_ALTI[f'layer{layer}'].append(avg_precision(layer_importance, CI))
        
    ### Dot Product
    # Remove mask_index and then normalize the scores.
    ALTI_scores = ALTI_per_layer_scores[i][:, batch_sample['target_index']]
#     ALTI_scores = torch.from_numpy(ALTI_scores)
    ALTI_scores = torch.concat((ALTI_scores[:, :mask_index], ALTI_scores[:, mask_index + 1:]), dim=1)
    ALTI_scores = ALTI_scores / ALTI_scores.sum(dim=-1, keepdim=True)
    
    if CI[-1] > mask_index:
        CI_scores = ALTI_scores[:, CI - 1] # Because of the removed mask_index
    else:
        CI_scores = ALTI_scores[:, CI]
    
    # In case there are more than one cue indices (i.e. evidence)
    if CI.shape[0] > 1:
        CI_scores = CI_scores.sum(axis=1, keepdim=True)
        
    sum_ALTI_scores += CI_scores

print("### Dot Product ###")
print(sum_ALTI_scores / len(eval_dataset))

print("### Average Precision ###")
temp_list = list()
for layer in diagram_layers:
    temp_list.append(sum(APs_ALTI[f'layer{layer}']) / len(APs_ALTI[f'layer{layer}']))

print(temp_list)


  0%|          | 0/2208 [00:00<?, ?it/s]

### Dot Product ###
tensor([[0.1008],
        [0.1513],
        [0.1387],
        [0.1094],
        [0.1243],
        [0.1329],
        [0.1418],
        [0.1130],
        [0.1555],
        [0.1157],
        [0.1642],
        [0.1291]])
### Average Precision ###
[0.25378783982724146, 0.3765116972267928, 0.384735210937058, 0.3131666564541012, 0.32833106186536964, 0.3317546323066029, 0.35110838518617177, 0.30322958040349385, 0.4014851593655965, 0.32560539946549477, 0.4042661331723848, 0.3256675418105456]


In [ ]:
# RoBERTa pre-trained

### Dot Product ###
[0.1000, 0.1535, 0.1346, 0.1101, 0.1351, 0.1429, 0.1394, 0.1363, 0.1353, 0.1239, 0.1443, 0.1748]
### Average Precision ###
[0.24847647621734462, 0.3781740227312561, 0.38813648741068457, 0.30460218191646077, 0.37426771660807046, 0.374563165041807,
 0.3504273337748207, 0.3617342698439846, 0.3520740295961088, 0.32868386991179194, 0.37172958476457274, 0.39374281351833057]

# RoBERTa fine-tuned

### Dot Product ###
[0.1008, 0.1513, 0.1387, 0.1094, 0.1243, 0.1329, 0.1418, 0.1130, 0.1555, 0.1157, 0.1642, 0.1291]
### Average Precision ###
[0.25378783982724146, 0.37580808697834556, 0.384735210937058, 0.3131100441352606, 0.32833106186536964, 0.3317546323066029,
 0.35110838518617177, 0.30322958040349385, 0.4014851593655965, 0.3255487871466542, 0.4042661331723848, 0.3256675418105456]